In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()
import plotly.graph_objects as go
from copy import deepcopy
from stock_env.envs import *
from stock_env.algos.buffer import RolloutBuffer
from stock_env.algos.agent import MetaAgent
import gymnasium as gym
import torch as th
def plot_mean_reward(fig, df, fill_color, line_color, name):

    fig.add_trace(
        go.Scatter(
            x=df["n_adapt_steps"],
            y=df["mean-std"],
            mode="lines",
            fill=None,
            line=dict(width=0.5, color=fill_color),
            showlegend=False,
        )
    )

    fig.add_trace(
        go.Scatter(
            x=df["n_adapt_steps"],
            y=df["mean+std"],
            fill="tonexty",
            mode="lines",
            line=dict(width=0.5, color=fill_color),
            showlegend=False,
        )
    )

    fig.add_trace(
        go.Scatter(
            x=df["n_adapt_steps"],
            y=df["mean"],
            fill=None,
            line_color=line_color,
            name=name,
        )
    )

In [2]:
ARGS_PATH = "../configs/maml.yaml"
ENV_ID = "SP500-v0"
args = open_config(ARGS_PATH, env_id=ENV_ID, is_args=True)

In [3]:
envs = MetaVectorEnv([lambda: gym.make(ENV_ID) for _ in range(args.num_tasks)])
tasks = envs.sample_task(args.num_tasks)
for task, env in zip(tasks, envs.envs):
    env.reset_task(task)

In [4]:
import pandas as pd
obs, info = envs.reset()
info_df = pd.DataFrame(info)[['from_time', 'episode_ticker']]
obs_df = pd.DataFrame(obs)
df = pd.concat([info_df, obs_df], axis=1)
df = df.rename(columns={'from_time': 'time', 'episode_ticker': 'ticker'})
steps = args.num_steps // args.num_tasks
tickers = df['ticker']
for _ in range(steps):
    obs, reward, termninated, truncated, info = envs.step(envs.action_space.sample())
    _info_df = pd.DataFrame(info)[['time']]
    _info_df['ticker'] = tickers
    _obs_df = pd.DataFrame(obs)
    _df = pd.concat([_info_df, _obs_df], axis=1)
    df = df.append(_df, ignore_index=True)
df.to_csv('../temp/evaluate_buffer.csv', index=True)
del df

In [5]:
df = pd.read_csv('../temp/evaluate_buffer.csv', index_col=0)
agent = MetaAgent(envs, hiddens=args.hiddens)
batch = df.loc[:, slice('0', '149')].to_numpy()
batch = th.from_numpy(batch).float()
values = agent.get_value(batch)
values = values.detach().flatten().numpy()
df['values'] = values

In [7]:
df.columns[-1]

'149'